資料來源：[開放台灣民間租屋資料](https://rentalhouse.g0v.ddio.io/download)

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
taipei_houserent_df = pd.read_json('/Users/john.tung/Downloads/tw-rental-data/2019Q1-raw_台北市.json').set_index('vendor_house_id')
print(taipei_houserent_df.info())
taipei_houserent_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 65773 entries, 7443057 to 6612761
Data columns (total 72 columns):
additional_fee_cable_tv             65773 non-null bool
additional_fee_eletricity           65773 non-null bool
additional_fee_gas                  65773 non-null bool
additional_fee_internet             65773 non-null bool
additional_fee_water                65773 non-null bool
agent_org                           43548 non-null object
allow_pet                           63780 non-null float64
apt_feature_code                    38377 non-null object
author                              65762 non-null object
building_type                       63782 non-null float64
can_cook                            63780 non-null float64
contact                             65773 non-null int64
created                             65773 non-null object
deal_status                         65773 non-null int64
deal_time                           11068 non-null datetime64[ns, UTC]
deposit  

,additional_fee_cable_tv,additional_fee_eletricity,additional_fee_gas,additional_fee_internet,additional_fee_water,agent_org,allow_pet,apt_feature_code,author,building_type,...,top_region,total_floor,transportation_bus,transportation_hsr,transportation_public_bike,transportation_subway,transportation_train,updated,vendor,vendor_house_url
vendor_house_id,,,,,,,,,,,,,,,,,,,,,
7443057,True,True,True,True,True,台灣房屋,1.0,_01010202,e3fa3502-476f-4953-b982-c6541d95612b,0.0,...,17,5.0,1.0,0.0,0.0,1.0,1.0,2019-03-31T06:33:31.285Z,591 租屋網,https://rent.591.com.tw/rent-detail-7443057.html
7513817,True,True,True,False,False,None,0.0,None,fd561368-1a91-4ee2-bded-166cd058a0ec,0.0,...,17,4.0,NaN,NaN,NaN,NaN,NaN,2019-03-31T06:30:11.629Z,591 租屋網,https://rent.591.com.tw/rent-detail-7513817.html
7513897,True,True,True,True,True,None,0.0,_04020201,59681355-ad89-41b5-a712-cb25d99a5062,2.0,...,17,15.0,2.0,0.0,0.0,1.0,0.0,2019-03-31T06:30:18.216Z,591 租屋網,https://rent.591.com.tw/rent-detail-7513897.html
7512995,True,True,True,False,True,None,0.0,None,ff9cd36f-2e82-4387-8401-dd6228d84251,0.0,...,17,5.0,0.0,0.0,0.0,1.0,0.0,2019-03-31T06:30:00.389Z,591 租屋網,https://rent.591.com.tw/rent-detail-7512995.html
7513901,False,False,False,False,False,None,0.0,None,59681355-ad89-41b5-a712-cb25d99a5062,2.0,...,17,12.0,2.0,0.0,0.0,1.0,0.0,2019-03-31T06:29:54.642Z,591 租屋網,https://rent.591.com.tw/rent-detail-7513901.html


看資料分布，50% 和 max 差距不應該相差非常巨大，可以判斷有 Outlier，使用最簡單的超出三倍標準差概念將這些Outlier先去掉，避免對Model造成影響

In [3]:
taipei_houserent_df.describe()

,allow_pet,building_type,can_cook,contact,deal_status,deposit,deposit_type,dist_to_highest_floor,facilities_冰箱,facilities_冷氣,...,rough_coordinate_x,rough_coordinate_y,sub_region,top_region,total_floor,transportation_bus,transportation_hsr,transportation_public_bike,transportation_subway,transportation_train
count,63780.000000,63782.000000,63780.000000,65773.000000,65773.000000,5.918600e+04,65773.000000,63782.000000,61884.000000,61884.000000,...,65009.000000,65009.000000,65773.000000,65773.0,65772.000000,44982.000000,44982.0,44982.000000,44982.000000,44982.000000
mean,0.248432,1.372033,0.786877,1.388123,1.004181,7.512926e+04,0.208338,3.936644,0.817320,0.969621,...,25.055613,121.542452,1704.371840,17.0,9.114243,0.891201,0.0,0.021253,1.154773,0.155307
std,0.432107,0.915257,0.409517,0.886028,0.583139,7.925643e+04,0.610154,4.000265,0.386407,0.171630,...,0.030168,0.477744,3.591082,0.0,5.392538,1.234094,0.0,0.156358,0.518922,0.381453
min,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000e+02,0.000000,-1.000000,0.000000,0.000000,...,24.960111,0.000129,1700.000000,17.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
25%,0.000000,0.000000,1.000000,0.000000,1.000000,2.600000e+04,0.000000,1.000000,1.000000,1.000000,...,25.036120,121.524100,1701.000000,17.0,5.000000,0.000000,0.0,0.000000,1.000000,0.000000
50%,0.000000,2.000000,1.000000,2.000000,1.000000,5.400000e+04,0.000000,3.000000,1.000000,1.000000,...,25.051313,121.539062,1704.000000,17.0,8.000000,0.000000,0.0,0.000000,1.000000,0.000000
75%,0.000000,2.000000,1.000000,2.000000,1.000000,9.598600e+04,0.000000,6.000000,1.000000,1.000000,...,25.067610,121.563528,1708.000000,17.0,13.000000,1.000000,0.0,0.000000,1.000000,0.000000
max,1.000000,6.000000,1.000000,2.000000,2.000000,5.200000e+06,3.000000,70.000000,1.000000,1.000000,...,25.856747,123.985746,1711.000000,17.0,77.000000,8.000000,0.0,6.000000,4.000000,4.000000


In [7]:
# 找出最大值和 50% 相差過大的欄位
for column_name in taipei_houserent_df.describe().columns:
    if taipei_houserent_df.describe()[column_name]['max'] - taipei_houserent_df.describe()[column_name]['50%'] > 10:
        print(column_name)

deposit
dist_to_highest_floor
floor
floor_ping
monthly_management_fee
monthly_price
n_day_deal
per_ping_price
total_floor
